<a href="https://colab.research.google.com/github/SantiagoAlvarezb/Thesis/blob/main/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#For visualization and data manipulation
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
from matplotlib.animation import FuncAnimation
from IPython import display
import os
import random
import tensorflow as tf
import tensorboard
import torch.optim as optim
from torch import nn
import plotly.express as px

# Import Data

We import the needed datasets from kaggle. This includes the dataset corresponding to the play data as well as the three seasons of tracking data.

In [ ]:
#Define the path for each excel document that contains data
plays_path = '/content/drive/MyDrive/Enhancing Performance in Special Teams within the NFL through Reinforcement Learning: A Data-Driven Approach/nfl-big-data-bowl-2022/plays.csv'
tracking_2018_path  = '/content/drive/MyDrive/Enhancing Performance in Special Teams within the NFL through Reinforcement Learning: A Data-Driven Approach/nfl-big-data-bowl-2022/tracking2018.csv'
tracking_2019_path  = '/content/drive/MyDrive/Enhancing Performance in Special Teams within the NFL through Reinforcement Learning: A Data-Driven Approach/nfl-big-data-bowl-2022/tracking2019.csv'
tracking_2020_path  = '/content/drive/MyDrive/Enhancing Performance in Special Teams within the NFL through Reinforcement Learning: A Data-Driven Approach/nfl-big-data-bowl-2022/tracking2020.csv'

In [ ]:
#Import plays
plays = pd.read_csv(plays_path)

#Import tracking datasets
tracking_2018 = pd.read_csv(tracking_2018_path)
tracking_2019 = pd.read_csv(tracking_2019_path)
tracking_2020 = pd.read_csv(tracking_2020_path)

# Data Cleaning

First we look at the event distribution of our wanted plays.

In [ ]:
import plotly.graph_objects as go

# Define the colors for the bars
colors = {
    '2018': 'rgb(162, 9, 165)',  # Red
    '2019': 'rgb(236, 75, 39)',  # Green
    '2020': 'rgb(247, 164, 26)'   # Blue
}

# Filter the datasets for specific events
events = ['punt', 'kickoff']
tracking_datasets = [tracking_2018, tracking_2019, tracking_2020]
seasons = ['2018', '2019', '2020']

fig = go.Figure()

for i, season in enumerate(seasons):
    # Filter the dataset for the current season and selected events
    filtered_data = tracking_datasets[i][tracking_datasets[i]['event'].isin(events)]

    event_counts = filtered_data['event'].value_counts()
    fig.add_trace(go.Bar(
        x=event_counts.index,
        y=event_counts.values,
        name=season,
        marker=dict(color=colors[season])  # Set the color for the current season
    ))

fig.update_layout(
    title='Event Frequency by Season',
    xaxis_title='Event',
    yaxis_title='Frequency',
    barmode='group',
    bargap=0.2,  # Adjust the gap between bars within the same group
    bargroupgap=0.4  # Adjust the gap between bar groups
)

fig.show()


In this section we will focus on the datacleaning of the datasets. The process is as follows: since the tracking data is centered about the play data, the latter will be cleaned to take into account only plays whih matter to the study, meaning punts and kickoffs that are returned.

In [ ]:
#Filter out only Punts and Kickoffs which are returned
kickoffs_punts = ["Kickoff", "Punt"]
returns = ["Return"]
plays = plays.loc[plays['specialTeamsPlayType'].isin(kickoffs_punts)]
plays = plays.loc[plays['specialTeamsResult'].isin(returns)]

#Drop uncesesarry columns
plays.drop(['yardsToGo','kickerId', 'returnerId', 'kickBlockerId', 'yardlineSide','yardlineNumber', 'penaltyJerseyNumbers','penaltyYards', 'passResult', 'preSnapHomeScore', 'preSnapVisitorScore'], axis=1, inplace=True)

#Identify penalties
penalty_values = plays["penaltyCodes"].unique()
index = [1]
penalties = np.delete(penalty_values, index)

#Eliminate observations with penalties
plays = plays[plays.penaltyCodes.isin(penalties) == False]

#Eliminate columns
plays.drop(["penaltyCodes"], axis=1, inplace=True)

#Eliminate null values
plays.dropna(subset=['kickReturnYardage'], inplace=True)

#Take out onside kicks
onside_kicks = [(2018090903, 4162), (2018091601, 4214), (2018091700, 4233), (2018092303, 3993), (2018100400, 4630), (2018100706, 3861), (2018100708, 3388), (2018100709, 3866), (2018100710, 4640), (2018101406, 4443), (2018102100, 3461), (2018102101, 4147), (2018102200, 4160), (2018102500, 3136), (2018102805, 4119), (2018102806, 4252), (2018110401, 4373), (2018110405, 3539), (2018110408, 3916), (2018111100, 3841), (2018111110, 3192), (2018111802, 4588), (2018111803, 3993), (2018111805, 3972), (2018112201, 4254), (2018112202, 4080), (2018112501, 4757), (2018120202, 4188), (2018120204, 4105), (2018120210, 4744), (2018120903, 4536), (2018121000, 3920), (2018122306, 4140), (2018122308, 4209), (2018123013, 36), (2019090800, 4512), (2019090802, 3292), (2019090802, 3847), (2019091504, 3954), (2019092206, 3536), (2019100610, 4149), (2019100612, 4473), (2019101305, 4223), (2019101310, 4192), (2019101311, 3498), (2019102001, 4109), (2019102002, 2335), (2019102002, 4765), (2019102700, 3814), (2019110302, 4056), (2019110304, 4382), (2019111702, 2281), (2019112400, 4654), (2019112408, 3699), (2019112411, 4247), (2019112801, 4086), (2019120106, 2243), (2019120106, 4291), (2019120111, 3957), (2019120500, 4801), (2019120800, 3719), (2019120802, 4377), (2019120803, 3768), (2019120805, 4054), (2019121200, 3229), (2019121500, 3721), (2019121507, 3855), (2019121509, 4220), (2019122201, 3707), (2019122901, 4382), (2020091000, 3649), (2020091302, 4574), (2020091310, 4253), (2020091400, 3765), (2020091700, 4638), (2020092003, 3697), (2020092004, 2175), (2020092004, 4266), (2020092008, 3994), (2020092704, 4048), (2020100401, 4415), (2020100402, 4288), (2020100406, 3880), (2020100409, 3742), (2020100412, 4022), (2020101100, 4061), (2020101802, 3709), (2020101803, 3806), (2020101803, 4096), (2020101805, 4535), (2020102509, 3421), (2020110100, 2624), (2020110109, 3822), (2020110109, 4412), (2020110200, 4374), (2020110800, 3988), (2020110800, 4321), (2020110803, 4079), (2020110804, 4196), (2020110806, 4166), (2020111507, 3876), (2020111509, 4198), (2020111512, 3726), (2020112205, 250), (2020112903, 4350), (2020112911, 3867), (2020120607, 4318), (2020120607, 4668), (2020120608, 4566), (2020120700, 4251), (2020120702, 2370), (2020120800, 3611), (2020121305, 4406), (2020121900, 4262), (2020122004, 3506), (2020122013, 4543), (2020122500, 4187), (2020122602, 3928), (2020122710, 4234), (2021010300, 4494), (2021010304, 3985), (2021010315, 4182)]
for i in onside_kicks:
  plays=plays[~((plays['gameId'] == i[0]) & (plays['playId'] == i[1]))]

#Pairs
pairs = list(zip(plays['gameId'],plays['playId']))

For the datasets that contain the tracking data we are going to filter out such that only the plays that are present in the previous dataset appear.

In [ ]:
# Extract the game IDs and play IDs from the pairs
game_ids = [pair[0] for pair in pairs]
play_ids = [pair[1] for pair in pairs]

In [ ]:
tracking_2018 = tracking_2018[tracking_2018['gameId'].isin(game_ids) & tracking_2018['playId'].isin(play_ids)]
tracking_2019 = tracking_2019[tracking_2019['gameId'].isin(game_ids) & tracking_2019['playId'].isin(play_ids)]
tracking_2020 = tracking_2020[tracking_2020['gameId'].isin(game_ids) & tracking_2020['playId'].isin(play_ids)]

In [ ]:
print("Shape of 2018 tracking dataframe:", tracking_2018.shape)
print("Shape of 2019 tracking dataframe:", tracking_2019.shape)
print("Shape of 2020 tracking dataframe:", tracking_2020.shape)

Shape of 2018 tracking dataframe: (9397432, 18)
Shape of 2019 tracking dataframe: (8992977, 18)
Shape of 2020 tracking dataframe: (8893295, 18)


In [ ]:
pairs_2018 = list(set(zip(tracking_2018['gameId'], tracking_2018['playId'])))
pairs_2019 = list(set(zip(tracking_2019['gameId'], tracking_2019['playId'])))
pairs_2020 = list(set(zip(tracking_2020['gameId'], tracking_2020['playId'])))

In [ ]:
pairs = pairs_2018 + pairs_2019 + pairs_2020

Lastly, we once again generate the image to see the difference.

In [ ]:
import plotly.graph_objects as go

# Define the colors for the bars
colors = {
    '2018': 'rgb(162, 9, 165)',  # Red
    '2019': 'rgb(236, 75, 39)',  # Green
    '2020': 'rgb(247, 164, 26)'   # Blue
}

# Filter the datasets for specific events
events = ['punt', 'kickoff']
tracking_datasets = [tracking_2018, tracking_2019, tracking_2020]
seasons = ['2018', '2019', '2020']

fig = go.Figure()

for i, season in enumerate(seasons):
    # Filter the dataset for the current season and selected events
    filtered_data = tracking_datasets[i][tracking_datasets[i]['event'].isin(events)]

    event_counts = filtered_data['event'].value_counts()
    fig.add_trace(go.Bar(
        x=event_counts.index,
        y=event_counts.values,
        name=season,
        marker=dict(color=colors[season])  # Set the color for the current season
    ))

fig.update_layout(
    title='Event Frequency by Season',
    xaxis_title='Event',
    yaxis_title='Frequency',
    barmode='group',
    bargap=0.2,  # Adjust the gap between bars within the same group
    bargroupgap=0.4  # Adjust the gap between bar groups
)

fig.show()


# Save Clean Data

In [ ]:
tracking_2018.to_csv("/content/drive/MyDrive/Enhancing Performance in Special Teams within the NFL through Reinforcement Learning: A Data-Driven Approach/clean-data-nfl-big-data-bowl-2020/tracking2018.csv", index=False)
tracking_2019.to_csv("/content/drive/MyDrive/Enhancing Performance in Special Teams within the NFL through Reinforcement Learning: A Data-Driven Approach/clean-data-nfl-big-data-bowl-2020/tracking2019.csv", index=False)
tracking_2020.to_csv("/content/drive/MyDrive/Enhancing Performance in Special Teams within the NFL through Reinforcement Learning: A Data-Driven Approach/clean-data-nfl-big-data-bowl-2020/tracking2020.csv", index=False)